### Imports 

In [5]:
# Imports
import torch as pt
from torch import nn

print(f"Torch version: {pt.__version__}")

# if pt.cuda.is_available():
#     device = 'cuda'
# if pt.backends.mps.is_available():
#     device = 'mps'
# else:
#     device= 'cpu'
device = 'cpu'
print(f'device: {device}')

Torch version: 2.0.1
device: cpu


### Downloading a custom dataset

In [6]:
from pathlib import Path
import importLib
from sys import path
import zipfile


# Create directory
data_path = Path(f"{path[0]}/data")
image_path = data_path / 'pizza_steak_sushi'
if image_path.exists():
    print('Already exists')
else:
    image_path.mkdir(parents=True)


# Download pizza, steak and sushi data
# open skapar en zip fil som sedan fylls genom request
importLib.import_from_github('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip',directory=data_path)
with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip', 'r') as zip_ref:
    print('Unzipping pizza, steak and sushi data')
    zip_ref.extractall(image_path)
Path.unlink(data_path/'pizza_steak_sushi.zip')


Already exists
/Users/gustavgamstedt/Desktop/github to hemma/PyTorch/04/data/pizza_steak_sushi.zip doesn't exist, download
Unzipping pizza, steak and sushi data


In [7]:
# Setup training and testing paths
train_dir = image_path / 'train'
test_dir = image_path / 'test'

train_dir, test_dir

(PosixPath('/Users/gustavgamstedt/Desktop/github to hemma/PyTorch/04/data/pizza_steak_sushi/train'),
 PosixPath('/Users/gustavgamstedt/Desktop/github to hemma/PyTorch/04/data/pizza_steak_sushi/test'))

### Create dataset and dataloaders

In [8]:
from torchvision import transforms
simple_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

In [9]:
from torch.utils.data import DataLoader
from torchvision import datasets

train_dataset = datasets.ImageFolder(root = train_dir, transform=simple_transform)
test_dataset = datasets.ImageFolder(root = test_dir, transform=simple_transform)

In [10]:
import os
BATCH_SIZE = 32
NUM_WORKERS = round(os.cpu_count()*(3/4))
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    num_workers=NUM_WORKERS,
    shuffle=False
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    num_workers=NUM_WORKERS,
    shuffle=False
)

### Create model

In [19]:
from torch import nn

class ModelWithoutAugmentation(nn.Module):
    def __init__(self, input_features:int,output_features:int, hidden_units:int=10):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(input_features, hidden_units,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),


            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),


            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*256, output_features)
        )
    def forward(self, X:pt.Tensor) -> pt.Tensor:
        X_change = self.conv_block_1(X)
        X_change = self.conv_block_2(X_change)
        print(X_change.shape)
        X_change = self.classifier(X_change)
        return X_change

In [20]:
pt.manual_seed(42)
model0 = ModelWithoutAugmentation(input_features=3, output_features=len(train_dataset.classes), hidden_units=10).to(device)
model0

ModelWithoutAugmentation(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

#### Testing model with random data

In [26]:
imgs, labels = next(iter(train_dataloader))
print(imgs.shape, len(labels))
# model0(imgs[0].unsqueeze(0))
model0(imgs.to(device))

torch.Size([32, 3, 64, 64]) 32
torch.Size([32, 10, 16, 16])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x2560 and 1690x3)

### Summarize a model

In [22]:
try:
    import torchinfo
except ModuleNotFoundError:
    print('Module not found, installing module')
    !pip3 install torchinfo

In [23]:
torchinfo.summary(model0, input_size=[32,3,64,64],device=device)

torch.Size([32, 10, 16, 16])


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, Conv2d: 2, ReLU: 2, Conv2d: 2, ReLU: 2, MaxPool2d: 2, Sequential: 1, Conv2d: 2, ReLU: 2, Conv2d: 2, ReLU: 2, MaxPool2d: 2, Flatten: 2]